# Sheet

**0. Выгрузка данных**

- [Карточка датасета](https://www.kaggle.com/datasets/wordsforthewise/lending-club/data)
- [Репо с парсингом](https://github.com/nateGeorge/preprocess_lending_club_data)

~~PS кажется там везде данные до 2к18, придется парсить с сайта (для него нужен VPN)~~

- Хорошее [описание переменных](https://www.openintro.org/data/index.php?data=loans_full_schema) (там же ноут)
- И еще на [гите](https://github.com/dosei1/Lending-Club-Loan-Data/blob/master/LCDataDictionary.csv)
-  тоже [прикольный разбор](https://github.com/pawanreddy-u/lendingclub)
- что-то [умное](https://www.kaggle.com/code/pragyanbo/a-hitchhiker-s-guide-to-lending-club-loan-data#Modeling) с кегла для вдохновения

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os
import requests
import joblib

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from feature_engine.encoding import WoEEncoder
from tqdm import tqdm

In [2]:
data = pd.read_csv('data_selected_2.csv').drop(['recoveries', 'loan_status'], axis = 1)
data.head()

,loan_amnt,term,int_rate,installment,home_ownership,annual_inc,issue_d,pymnt_plan,dti,inq_last_6mths,...,num_tl_30dpd,pub_rec_bankruptcies,tot_hi_cred_lim,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag,loan_condition_int,loan_condition
0,3600.0,0,13.99,123.03,1,55000.0,2015-12,0,5.91,1,...,0,0.0,178050.0,2400.0,13734.0,0,0,0,0,1
1,24700.0,0,11.99,820.28,1,65000.0,2015-12,0,16.06,4,...,0,0.0,314017.0,79300.0,24667.0,0,0,0,0,1
2,20000.0,1,10.78,432.66,1,63000.0,2015-12,0,10.78,0,...,0,0.0,218418.0,6200.0,14877.0,0,0,0,0,1
3,35000.0,1,14.85,829.90,1,110000.0,2015-12,0,17.06,0,...,0,0.0,381215.0,62500.0,18000.0,0,0,0,0,1
4,10400.0,1,22.45,289.91,1,104433.0,2015-12,0,25.37,3,...,0,0.0,439570.0,20300.0,88097.0,0,0,0,0,1


In [3]:
cat_features = ['term', 'home_ownership', 'pymnt_plan', 'initial_list_status', 'application_type', 'hardship_flag', 'disbursement_method',\
 'debt_settlement_flag']
date_columns = ['issue_d', 'last_credit_pull_d', 'bc_open_to_buy']
num_features = list(set(data.columns).difference(set(cat_features+date_columns + ['loan_condition_int'])))
num_features.remove('tot_hi_cred_lim')
# num_features = ['num_il_tl', 'num_tl_30dpd', ]
for col in date_columns:
    data[col + "_year"] = pd.to_datetime(data[col]).map(lambda x: x.year)
    data[col + "_month"] = pd.to_datetime(data[col]).map(lambda x: x.month)


num_feature_type = {x: "float" for x in num_features}
cat_feature_type = {x: "str" for x in cat_features}

year_col = list(filter(lambda x: "_year" in x, data.columns))
year_feature_type = {x: "str" for x in year_col}

month_col = list(filter(lambda x: "_month" in x, data.columns))
month_feature_type = {x: "str" for x in month_col}


feature_columns = year_col + month_col +\
                 cat_features + num_features


features_type = dict(**num_feature_type,
                     **cat_feature_type,
                     **year_feature_type,
                     **month_feature_type)

In [4]:
data=data[feature_columns + ['loan_condition_int']].astype(features_type)

In [5]:
data.columns

Index(['issue_d_year', 'last_credit_pull_d_year', 'bc_open_to_buy_year',
       'issue_d_month', 'last_credit_pull_d_month', 'bc_open_to_buy_month',
       'term', 'home_ownership', 'pymnt_plan', 'initial_list_status',
       'application_type', 'hardship_flag', 'disbursement_method',
       'debt_settlement_flag', 'annual_inc', 'revol_util', 'int_rate',
       'mo_sin_old_il_acct', 'num_tl_120dpd_2m', 'inq_last_6mths', 'loan_amnt',
       'total_rev_hi_lim', 'total_il_high_credit_limit', 'avg_cur_bal',
       'num_il_tl', 'installment', 'pub_rec_bankruptcies', 'total_acc',
       'mort_acc', 'num_bc_tl', 'dti', 'mo_sin_old_rev_tl_op',
       'acc_now_delinq', 'mths_since_recent_bc', 'revol_bal', 'total_bc_limit',
       'num_tl_30dpd', 'loan_condition', 'loan_condition_int'],
      dtype='object')

In [6]:
last_train_date = '2018'
idx_train = (data['issue_d_year'] < last_train_date)

train = data.loc[idx_train, feature_columns + ['loan_condition_int']].copy()
test = data.loc[~idx_train, feature_columns + ['loan_condition_int']].copy()

print(train.shape, test.shape)

(1695251, 39) (495242, 39)


Делаем бинаризацию числовых фичей

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
from scipy import stats

In [8]:
class Binning(BaseEstimator, TransformerMixin):

    def __init__(self, y, n_threshold, y_threshold, p_threshold, sign=False):
        self.n_threshold = n_threshold
        self.y_threshold = y_threshold
        self.p_threshold = p_threshold
        self.y = y
        self.sign = sign

        self.init_summary = pd.DataFrame()
        self.bin_summary = pd.DataFrame()
        self.pvalue_summary = pd.DataFrame()
        self.dataset = pd.DataFrame()
        self.woe_summary = pd.DataFrame()

        self.column = object
        self.total_iv = object
        self.bins = object
        self.bucket = object

    def generate_summary(self):

        self.init_summary = self.dataset.groupby([self.column]).agg({self.y: ["mean", "std", "size"]})\
        .rename({"mean": "means", "size": "nsamples", "std": "std_dev"}, axis=1)

        self.init_summary.columns = self.init_summary.columns.droplevel(level=0)

        self.init_summary = self.init_summary[["means", "nsamples", "std_dev"]]
        self.init_summary = self.init_summary.reset_index()

        self.init_summary["del_flag"] = 0
        self.init_summary["std_dev"] = self.init_summary["std_dev"].fillna(0)

        self.init_summary = self.init_summary.sort_values([self.column], ascending=self.sign)

    def combine_bins(self):
        summary = self.init_summary.copy()

        while True:
            i = 0
            summary = summary[summary.del_flag != 1]
            summary = summary.reset_index(drop=True)
            while True:

                j = i + 1

                if j >= len(summary):
                    break

                if summary.iloc[j].means < summary.iloc[i].means:
                    i = i + 1
                    continue
                else:
                    while True:
                        n = summary.iloc[j].nsamples + summary.iloc[i].nsamples
                        m = (summary.iloc[j].nsamples * summary.iloc[j].means +
                             summary.iloc[i].nsamples * summary.iloc[i].means) / n

                        if n == 2:
                            s = np.std([summary.iloc[j].means, summary.iloc[i].means])
                        else:
                            s = np.sqrt((summary.iloc[j].nsamples * (summary.iloc[j].std_dev ** 2) +
                                         summary.iloc[i].nsamples * (summary.iloc[i].std_dev ** 2)) / n)

                        summary.loc[i, "nsamples"] = n
                        summary.loc[i, "means"] = m
                        summary.loc[i, "std_dev"] = s
                        summary.loc[j, "del_flag"] = 1

                        j = j + 1
                        if j >= len(summary):
                            break
                        if summary.loc[j, "means"] < summary.loc[i, "means"]:
                            i = j
                            break
                if j >= len(summary):
                    break
            dels = np.sum(summary["del_flag"])
            if dels == 0:
                break

        self.bin_summary = summary.copy()

    def calculate_pvalues(self):
        summary = self.bin_summary.copy()
        while True:
            summary["means_lead"] = summary["means"].shift(-1)
            summary["nsamples_lead"] = summary["nsamples"].shift(-1)
            summary["std_dev_lead"] = summary["std_dev"].shift(-1)

            summary["est_nsamples"] = summary["nsamples_lead"] + summary["nsamples"]
            summary["est_means"] = (summary["means_lead"] * summary["nsamples_lead"] +
                                    summary["means"] * summary["nsamples"]) / summary["est_nsamples"]

            summary["est_std_dev2"] = (summary["nsamples_lead"] * summary["std_dev_lead"] ** 2 +
                                       summary["nsamples"] * summary["std_dev"] ** 2) / (summary["est_nsamples"] - 2)

            summary["z_value"] = (summary["means"] - summary["means_lead"]) / np.sqrt(
                summary["est_std_dev2"] * (1 / summary["nsamples"] + 1 / summary["nsamples_lead"]))

            summary["p_value"] = 1 - stats.norm.cdf(summary["z_value"])

            summary["p_value"] = summary.apply(
                lambda row: row["p_value"] + 1 if (row["nsamples"] < self.n_threshold) |
                                                  (row["nsamples_lead"] < self.n_threshold) |
                                                  (row["means"] * row["nsamples"] < self.y_threshold) |
                                                  (row["means_lead"] * row["nsamples_lead"] < self.y_threshold)
                else row["p_value"], axis=1)

            max_p = max(summary["p_value"])
            row_of_maxp = summary['p_value'].idxmax()
            row_delete = row_of_maxp + 1

            if max_p > self.p_threshold:
                summary = summary.drop(summary.index[row_delete])
                summary = summary.reset_index(drop=True)
            else:
                break

            summary["means"] = summary.apply(lambda row: row["est_means"] if row["p_value"] == max_p else row["means"],
                                             axis=1)
            summary["nsamples"] = summary.apply(
                lambda row: row["est_nsamples"] if row["p_value"] == max_p else row["nsamples"], axis=1)
            summary["std_dev"] = summary.apply(
                lambda row: np.sqrt(row["est_std_dev2"]) if row["p_value"] == max_p else row["std_dev"], axis=1)

        self.pvalue_summary = summary.copy()

    def calculate_woe(self):
        woe_summary = self.pvalue_summary[[self.column, "nsamples", "means"]]

        woe_summary["bads"] = woe_summary["means"] * woe_summary["nsamples"]
        woe_summary["goods"] = woe_summary["nsamples"] - woe_summary["bads"]

        total_goods = np.sum(woe_summary["goods"])
        total_bads = np.sum(woe_summary["bads"])

        woe_summary["dist_good"] = woe_summary["goods"] / total_goods
        woe_summary["dist_bad"] = woe_summary["bads"] / total_bads

        woe_summary["WOE_" + self.column] = np.log(woe_summary["dist_good"] / woe_summary["dist_bad"])

        woe_summary["IV_components"] = (woe_summary["dist_good"] - woe_summary["dist_bad"]) * woe_summary[
            "WOE_" + self.column]

        self.total_iv = np.sum(woe_summary["IV_components"])
        self.woe_summary = woe_summary

    def generate_bin_labels(self,row):
        return "-".join(map(str, np.sort([row[self.column], row[self.column + "_shift"]])))

    def generate_final_dataset(self):
            
        if self.sign == False:
            shift_var = 1
            self.bucket = True
        else:
            shift_var = -1
            self.bucket = False

        self.woe_summary[self.column + "_shift"] = self.woe_summary[self.column].shift(shift_var)

        if self.sign == False:
            self.woe_summary.loc[0, self.column + "_shift"] = -np.inf
            self.bins = np.sort(list(self.woe_summary[self.column]) + [np.Inf,-np.Inf])
        else:
            self.woe_summary.loc[len(self.woe_summary) - 1, self.column + "_shift"] = np.inf
            self.bins = np.sort(list(self.woe_summary[self.column]) + [np.Inf,-np.Inf])

        self.woe_summary['n_bin'] = self.woe_summary[self.column].rank().astype('int')
        self.dataset['n_bin'] = np.digitize(self.dataset[self.column], self.bins, right = self.bucket).astype('int')
        if sum(self.dataset['n_bin'] == 1) == 0:
            self.bins  = np.delete(self.bins, 1)

        if sum(self.dataset['n_bin'] == len(self.bins)-1) == 0:
            self.bins  = np.delete(self.bins, 1)
            
        self.dataset['n_bin'] = np.digitize(self.dataset[self.column], self.bins, right = self.bucket).astype('int')
        self.woe_summary["labels"] = self.woe_summary.apply(self.generate_bin_labels, axis=1)

        self.dataset["bins"] = pd.cut(self.dataset[self.column], self.bins, right=self.bucket, precision=0)
        self.dataset["bins"] = self.dataset["bins"].astype(str)
        self.dataset['bins'] = self.dataset['bins'].map(lambda x: x.lstrip('[').rstrip(')'))

        self.dataset = pd.merge(self.dataset, self.woe_summary[['n_bin', 'WOE_' + self.column]], on = 'n_bin', how = 'left')

    def fit(self, dataset):
        self.dataset = dataset
        self.column = self.dataset.columns[self.dataset.columns != self.y][0]

        self.generate_summary()
        self.combine_bins()
        self.calculate_pvalues()
        self.calculate_woe()
        self.generate_final_dataset()

    def transform(self, test_data):
        test_data[self.column+"_bins"] = pd.cut(test_data[self.column], self.bins, right=self.bucket, precision=0)
        #добавил
        test_data["n_bin"] = np.digitize(test_data[self.column], self.bins, right = self.bucket).astype('int')
        test_data = pd.merge(test_data, self.woe_summary[['n_bin', 'WOE_' + self.column]], on = 'n_bin', how = 'left')
        # test_data.drop(columns = ['n_bin'], inplace = True)
        return test_data

выяснял знак для каждой колонки

In [32]:
# from tqdm import tqdm
# y_var = 'loan_condition_int'
# sign = False

# dct = {}


# for feature in tqdm(num_features):
#     # sign = True if feature in true_sign else False
#     # print(feature, sign)
#     bin_object = Binning(y_var, n_threshold = 50, y_threshold = 10, p_threshold = 0.35, sign=sign)
#     bin_object.fit(train[[y_var, feature]])
#     bins = len(bin_object.woe_summary)
#     dct.update({feature: bins})
#     # if bins < 3:
#     #     bin_object1 = Binning(y_var, n_threshold = 50, y_threshold = 10, p_threshold = 0.35, sign=True)
#     #     bin_object1.fit(train[[y_var, feature]])
#     #     print(feature, bins, len(bin_object1.woe_summary))
#     #     if len(bin_object1.woe_summary) > bins:
#     #         train = bin_object1.transform(train[[y_var] + feature_columns])
#     #         test = bin_object1.transform(test[[y_var] + feature_columns])
#     #         continue
#     # train = bin_object.transform(train)
#     # test = bin_object.transform(test)

In [17]:
sorted(dct.items(), key=lambda item: item[1])

[('loan_condition', 1),
 ('total_bc_limit', 1),
 ('mths_since_recent_bc', 1),
 ('bc_open_to_buy', 1),
 ('tot_hi_cred_lim', 1),
 ('mort_acc', 1),
 ('revol_bal', 2),
 ('total_il_high_credit_limit', 2),
 ('avg_cur_bal', 2),
 ('total_rev_hi_lim', 2),
 ('mo_sin_old_rev_tl_op', 3),
 ('num_tl_120dpd_2m', 3),
 ('acc_now_delinq', 4),
 ('num_tl_30dpd', 4),
 ('annual_inc', 4),
 ('num_bc_tl', 5),
 ('inq_last_6mths', 5),
 ('pub_rec_bankruptcies', 5),
 ('mo_sin_old_il_acct', 7),
 ('total_acc', 10),
 ('loan_amnt', 14),
 ('num_il_tl', 14),
 ('installment', 26),
 ('revol_util', 27),
 ('int_rate', 32),
 ('dti', 36)]

In [9]:
import warnings
warnings.filterwarnings('ignore')

Обработка всех фичей

In [10]:
class Process_all_features:
    def __init__(self, num_features, cat_features, feature_columns, fillna = True, target_col = 'loan_condition_int'):
        
        self.false_sign = ['dti', 'int_rate', 'revol_util', 'installment', 'num_il_tl', 'loan_amnt', 'total_acc',\
             'mo_sin_old_il_acct', 'pub_rec_bankruptcies', 'inq_last_6mths', 'num_bc_tl', 'annual_inc',
             'num_tl_30dpd', 'acc_now_delinq', 'num_tl_120dpd_2m', 'loan_condition']
        
        self.true_sign = ['mo_sin_old_rev_tl_op','avg_cur_bal', 'total_bc_limit',  'total_il_high_credit_limit','mort_acc',
                         'bc_open_to_buy','mths_since_recent_bc', 'revol_bal', 'tot_hi_cred_lim', 'total_rev_hi_lim']
        
        self.num_features = num_features
        self.cat_features = cat_features
        self.feature_columns = feature_columns
        self.year_col = year_col
        self.month_col = month_col
        self.fillna = fillna
        self.target = target_col
        self.dct_bins = {}

    def preproc_num_features(self, train, test):
        print('process num_features..')
        for feature in tqdm(self.num_features):
            print(feature)
            if self.fillna:
                train[feature] = train[feature].fillna(train[feature].mean())
                test[feature] = test[feature].fillna(test[feature].mean())
            sign = True if feature in self.true_sign else False
            
            bin_object = Binning(self.target, n_threshold = 50, y_threshold = 10, p_threshold = 0.35, sign=sign)
            bin_object.fit(train[[self.target, feature]])
            bins = len(bin_object.woe_summary)
            self.dct_bins.update({feature: bins})
            # return bin_object
        
            train = bin_object.transform(train)
            test = bin_object.transform(test)
            
        new_num_features = list(map(lambda x:  'WOE_' + x, num_features))
        new_features =  self.year_col + self.month_col + self.cat_features + new_num_features
        self.features = new_features
        return train[new_features + [self.target]], test[new_features + [self.target]]

    def preproc_cat_features(self, train, test):
        print('process cat_features..')
        woe = WoEEncoder(variables = self.cat_features + self.year_col + self.month_col)
        X_train = woe.fit_transform(train.drop(columns = [self.target]), train[self.target]).add_suffix('_woe')
        X_test = woe.transform(test.drop(columns = [self.target])).add_suffix('_woe')
        return X_train, train[self.target], X_test, test[self.target]

    def preproc_all_features(self, train, test):
        train1 = train.copy()
        test1 = test.copy()
        # print(train.isna().sum())
        train1, test1 = self.preproc_num_features(train1, test1)
        # print(train1.isna().sum())
        X_train, y_train, X_test, y_test = self.preproc_cat_features(train1, test1)
        return X_train, y_train, X_test, y_test

In [11]:
PR = Process_all_features(num_features, cat_features, feature_columns)

In [ ]:
PR.dct_bins

In [12]:
X_train, y_train, X_test, y_test = PR.preproc_all_features(train, test)

process num_features..


  0%|                                                    | 0/24 [00:00<?, ?it/s]

annual_inc


  4%|█▊                                         | 1/24 [02:21<54:11, 141.39s/it]

revol_util


...

num_tl_30dpd


 96%|█████████████████████████████████████████▏ | 23/24 [19:38<00:40, 40.33s/it]

loan_condition


100%|███████████████████████████████████████████| 24/24 [19:43<00:00, 49.30s/it]


process cat_features..


In [16]:
X_test.isna().sum()

issue_d_year_woe                      495242
last_credit_pull_d_year_woe                0
bc_open_to_buy_year_woe                    0
issue_d_month_woe                          0
last_credit_pull_d_month_woe               0
bc_open_to_buy_month_woe                   0
term_woe                                   0
home_ownership_woe                         0
pymnt_plan_woe                             0
initial_list_status_woe                    0
application_type_woe                       0
hardship_flag_woe                          0
disbursement_method_woe                    0
debt_settlement_flag_woe                   0
WOE_annual_inc_woe                         0
WOE_revol_util_woe                         0
WOE_int_rate_woe                           0
WOE_mo_sin_old_il_acct_woe                 0
WOE_num_tl_120dpd_2m_woe                   0
WOE_inq_last_6mths_woe                     0
WOE_loan_amnt_woe                          0
WOE_total_rev_hi_lim_woe                   0
WOE_total_

In [ ]:
# !pip install scorecardpy

In [22]:

import scorecardpy as sc
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l1', C=0.9, solver='saga', n_jobs=-1)
lr.fit(X_train, y_train)
train_pred = lr.predict_proba(X_train)[:,1]
test_pred = lr.predict_proba(X_test)[:,1]

In [23]:
train_pred = lr.predict_proba(X_train)[:,1]
test_pred = lr.predict_proba(X_test)[:,1]

In [26]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train, train_pred)

0.8119919020121517

In [27]:
roc_auc_score(y_test, test_pred)

0.7027829313454489

In [ ]:
test_perf = sc.perf_eva(y_train, train_pred, title = "test")

In [ ]:
test_perf = sc.perf_eva(y_test, test_pred, title = "test")